In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/sample_data/River-and-Rail-Cantina.csv")

In [3]:
df['user_name'] = df['user_name'].astype(str)
df['rating'] = df['rating'].astype(str)
df['rating'] = df['rating'].str.replace(' star rating', '')
df['review_text'] = df['review_text'].astype(str)

In [4]:
df['user_name'] = df['user_name'].str.strip("['']")
df['rating'] = df['rating'].str.strip("['']")
df['review_text'] = df['review_text'].str.strip("['']")

In [5]:
df['rating'] = df['rating'].str.split("'", expand=True).get(0) #Some people reviewed the restaurant twice and I couldn't figure out how to split the reviews so I just deleted the second review
df['user_name'] = df['user_name'].str.split("'", expand=True).get(0) #Some people had the amount of photos posted so I cleaned it up
df['review_text'] = df['review_text'].str.split(", '", expand=True).get(0) #same as the ratings problem

In [6]:
df['rating'] = df['rating'].astype(int)
df.drop("Unnamed: 0", axis=1, inplace=True) #I do not know where unnamed:  0 came from so I dropped it

In [11]:
def recode_rating(rating):
    if rating <= 3:
        return 'Negative'
    elif rating <= 5:
        return 'Positive'
    else:
        return 'Unknown'

df['sentiment'] = df['rating'].apply(recode_rating)

In [12]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [14]:
def preprocess_review(review_text):
    # Remove punctuation
    review_text = review_text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert all words to lower case
    review_text = review_text.lower()
    
    # Tokenize the review into words
    words = word_tokenize(review_text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Join the remaining words back into a single string
    processed_review = ' '.join(words)
    
    return processed_review

df['review_text'] = df['review_text'].apply(preprocess_review)

In [15]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))

In [16]:
normalized_reviews = []
for review in df['review_text']:
    tokens = tokenizer.tokenize(review.lower())
    filtered_tokens = [token for token in tokens if token not in stop_words]
    normalized_reviews.append(filtered_tokens)

In [17]:

from gensim.models import Word2Vec

w2v_model = Word2Vec(normalized_reviews, vector_size=100, window=5, min_count=1, workers=4)

In [19]:
w2v_model.wv.most_similar(positive=['quesadillas'], topn=3)

[('hard', 0.9974542856216431),
 ('mexican', 0.9973438382148743),
 ('sit', 0.9973161220550537)]

The first two make sense but I do not really understand sit.